In [1]:
from numpy.random import seed
from tensorflow.random import set_seed
import random

seed(1)
set_seed(1)
random.seed(1)

import pandas as pd
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, InputLayer
from keras.callbacks import History 
from tensorflow.keras.optimizers import SGD, Adam

import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split, KFold, ParameterGrid
from sklearn.metrics import accuracy_score


from utils import k_fold, predict_and_accuracy, get_features

### Preprocessing

- Standardise numerical cols 
- Create new features


In [2]:
df = pd.read_csv('../data/Data_A.csv', header=None)

columns=['midprice_up','ss_lob_1_p','ss_lob_1_v','bs_lob_1_p','bs_lob_1_v','ss_lob_2_p','ss_lob_2_v',
'bs_lob_2_p','bs_lob_2_v','ss_lob_3_p','ss_lob_3_v','bs_lob_3_p','bs_lob_3_v','ss_lob_4_p','ss_lob_4_v',
'bs_lob_4_p','bs_lob_4_v','c1','c2','c3','c4','c5']

df.columns = columns

price_cols = ['ss_lob_1_p','bs_lob_1_p','ss_lob_2_p','bs_lob_2_p','ss_lob_3_p','bs_lob_3_p','ss_lob_4_p','bs_lob_4_p']
vol_cols = ['ss_lob_1_v','bs_lob_1_v','ss_lob_2_v','bs_lob_2_v','ss_lob_3_v','bs_lob_3_v','ss_lob_4_v','bs_lob_4_v']
prev_mid_m_cols = ['c1', 'c2', 'c3', 'c4', 'c5']


# engineer features
df, engineered_cols = get_features(df)


# do train test split
X = df.drop(columns='midprice_up')
y = df['midprice_up']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

means = X_train[price_cols+vol_cols+engineered_cols].mean(axis=0)
stds = X_train[price_cols+vol_cols+engineered_cols].std(axis=0)

def transform(df, means, stds):
    return (df-means)/stds

X_train[price_cols+vol_cols+engineered_cols] = transform(X_train[price_cols+vol_cols+engineered_cols], means, stds)
X_test[price_cols+vol_cols+engineered_cols] = transform(X_test[price_cols+vol_cols+engineered_cols], means, stds)



In [3]:
def get_default_model():
    model = keras.Sequential([
        InputLayer(input_shape=(len(X_train.columns))),
        Dense(units=100, activation='elu'),
        Dense(units=50, activation='elu'),
        Dense(units=20, activation='elu'),
        Dense(units=1, activation='sigmoid')
    ])
    history = History()

    return model, history

### Tune Epochs

In [4]:
for epochs in [5, 10,20]:
    print(f"{epochs=}")
    
    model, history = get_default_model()
    
    model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

    model.fit(X_train, y_train, batch_size=100, epochs=epochs, callbacks=[history], verbose=0)
    train_a, test_a = predict_and_accuracy(model,  X_train, X_test, y_train, y_test)
    print(f"train: {train_a:.4f}, test: {test_a:.4f}")



epochs=5
train: 0.7634, test: 0.7625
epochs=10
train: 0.7683, test: 0.7665
epochs=20
train: 0.7718, test: 0.7683


### Tune optimizer

In [5]:
for optimizer in [Adam(), SGD()]:

    print(f"{optimizer.get_config()['name']}")
    
    model, history = get_default_model()
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

    model.fit(X_train, y_train, batch_size=100, epochs=10, callbacks=[history], verbose=0)
    train_a, test_a = predict_and_accuracy(model,  X_train, X_test, y_train, y_test)
    print(f"train: {train_a:.4f}, test: {test_a:.4f}")


Adam
train: 0.7665, test: 0.7650
SGD
train: 0.7571, test: 0.7599


### Tune batch size

In [6]:
for batch_size in [50, 100, 500, 1000]:
    print(f"{batch_size=}")

    model, history = get_default_model()

    model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

    model.fit(X_train, y_train, batch_size=batch_size, epochs=10, callbacks=[history], verbose=0)
    train_a, test_a = predict_and_accuracy(model,  X_train, X_test, y_train, y_test)
    print(f"train: {train_a:.4f}, test: {test_a:.4f}\n")

batch_size=50
train: 0.7700, test: 0.7661

batch_size=100
train: 0.7677, test: 0.7640

batch_size=500
train: 0.7624, test: 0.7621

batch_size=1000
train: 0.7606, test: 0.7593



#### Tune Eta

In [7]:
for eta in [0.001, 0.003, 0.01, 0.05]:
    print(f"{eta=}")

    model, history = get_default_model()

    model.compile(optimizer=Adam(learning_rate=eta), loss="binary_crossentropy", metrics=["accuracy"])

    model.fit(X_train, y_train, batch_size=100, epochs=10, callbacks=[history], verbose=0)
    train_a, test_a = predict_and_accuracy(model,  X_train, X_test, y_train, y_test)
    print(f"train: {train_a:.4f}, test: {test_a:.4f}\n")

eta=0.001
train: 0.7683, test: 0.7662

eta=0.003
train: 0.7696, test: 0.7673

eta=0.01
train: 0.7643, test: 0.7651

eta=0.05
train: 0.5028, test: 0.5022



In [8]:
for units in [50, 100, 200]:
    print(f"{units=}")

    history = History()
    model = keras.Sequential([
        InputLayer(input_shape=(len(X_train.columns))),
        Dense(units=units, activation='elu'),
        Dense(units=1, activation='sigmoid')
    ])
    model = keras.models.clone_model(model)

    eta = 0.003
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=eta), loss="binary_crossentropy", metrics=["accuracy"])

    # k_fold(model, X_train, y_train, batch_size=100, epochs=5, history=history, splits=5)
    model.fit(X_train, y_train, batch_size=100, epochs=5, callbacks=[history], verbose=0)
    train_a, test_a = predict_and_accuracy(model,  X_train, X_test, y_train, y_test)
    print(f"train: {train_a:.4f}, test: {test_a:.4f}\n")

units=50
train: 0.7587, test: 0.7587

units=100
train: 0.7586, test: 0.7572

units=200
train: 0.7581, test: 0.7564

